<a href="https://colab.research.google.com/github/jhmendonca/notebooks/blob/main/GFPGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown # Connect Google Drive e baixar o GFPGAN
from IPython.utils import capture
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
import os
drive.mount('/content/gdrive')
if not os.path.exists("/content/gdrive/MyDrive/GFPGAN"):
  %cd /content/gdrive/MyDrive
  !git clone https://github.com/jhmendonca/GFPGAN.git
  %cd GFPGAN
  !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth -P experiments/pretrained_models
%cd /content/gdrive/MyDrive/GFPGAN

In [ ]:
#@markdown # Instalar Requerimentos
!pip install basicsr
!pip install facexlib
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan


In [ ]:
#@markdown # Upload das Fotos
import os
from google.colab import files
import shutil

upload_folder = 'inputs/upload'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.mkdir(upload_folder)


uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

In [ ]:
#@markdown # Realizar interferencia.
!rm -rf results
!python inference_gfpgan.py -i inputs/upload -o results -v 1.4 -s 2 --bg_upsampler realesrgan

In [ ]:
#@markdown # Visualizar resultados !
import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('GFPGAN output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# display each image in the upload folder
import os
import glob

input_folder = 'results/cropped_faces'
result_folder = 'results/restored_faces'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display(img_input, img_output)

In [ ]:
#@markdown # Baixar Fotos
!ls results
print('Download results')
os.system('zip -r download.zip results')
files.download("download.zip")